<a href="https://colab.research.google.com/github/emmayeh-0211/2021AI_ml/blob/main/titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from urllib.request import urlretrieve
url = "https://raw.githubusercontent.com/emmayeh-0211/2021AI_ml/main/train.csv"
urlretrieve(url, "train.csv")
url = "https://raw.githubusercontent.com/emmayeh-0211/2021AI_ml/main/test.csv"
urlretrieve(url, "test.csv")

('test.csv', <http.client.HTTPMessage at 0x7f9ecff65590>)

In [ ]:
import pandas as pd
train_df = pd.read_csv("train.csv", encoding="utf-8")
test_df = pd.read_csv("test.csv", encoding="utf-8")

In [ ]:
# 把訓練跟驗證的資料先合併，準備進行資料的前處理
# 不需要資料處理的欄位先drop掉
data = pd.concat([train_df, test_df], ignore_index=True)
data = data.drop(["PassengerId", "Survived"], axis=1)

In [ ]:
# 資料處理第一步 處理空值
# 把有空值的欄位篩選出來，進行資料前處理
na = data.isna().sum() #.isna() 尋找空值 .sum() 計算總數
# Series[帶入跟該列資料筆數一樣多的True/False list，來篩選要留下來的資料]
na[na > 0].sort_values(ascending=False) # .sort_values() 排列

Cabin       1014
Age          263
Embarked       2
Fare           1
dtype: int64

In [ ]:
# 資料前處理
# 先處理Cabin
def cabin_head(s):
  if not pd.isna(s):
    return s[0]

data['Cabin'] = data['Cabin'].apply(cabin_head)

In [ ]:
# Tickets 這邊要根據持有的票號，去算同一行人有多少人，然後把該數值放入tickets的資料內，意思是我們不看票號資訊，而是要新增一個資料是根據票號去算同行人數
# 因此先用 .value_counts() 去算各票號的次數（代表多少人持有同一個票號）
# 再將 data中 ticket的資訊替換成同一行人的數字（其實要取代或新增資料應該都是可以的）
dic = data['Ticket'].value_counts()
data['Ticket'] = data['Ticket'].apply(lambda t:dic[t])

In [ ]:
# 補缺失值(類別 -> 最常出現) [!，如果後續還有測試資料需要做資料前處理, 空值的部分不能重算, 直接補原本data中該欄位補的空值，不能大家各自補不同的空值]
most = data["Embarked"].value_counts().idxmax()
data["Embarked"] = data["Embarked"].fillna(most) # .fillna() 將空值補上括號()的值
na = data.isna().sum()
na[na > 0].sort_values(ascending=False)

Cabin    1014
Age       263
Fare        1
dtype: int64

In [ ]:
# 補缺失值（數值：中位數）
# .median() 查看各欄位的中位數
# 其中 Pclass艙等不是數值，是類別，因此將它從資料中drop掉
med = data.median().drop(['Pclass'])
data = data.fillna(med)  # 這裡為什麼不用指定空值要補在哪一欄裡？
na = data.isna().sum()
# Series[帶入跟該列資料筆數一樣多的True/False list，來篩選要留下來的資料]
na[na > 0].sort_values(ascending=False)

Cabin    1014
dtype: int64

In [ ]:
# 資料前處理，將名字的部分，只保留稱謂類型
def name_convert(name):
  name = name.split(',')[-1].split('.')[0]
  name = name.strip()
  return name
counts = data['Name'].apply(name_convert).value_counts()
whiltelist = counts[counts > 50].index
def name_convert(name):
  name = name.split(',')[-1].split('.')[0]
  name = name.strip()
  if name in whiltelist:
    return name
  else:
    return None
data['Name'] = data['Name'].apply(name_convert)

In [ ]:
# Step 2，進行one hot encoding
# 用.get_dummies()，就會自動偵測資料中字串中，就進行one hot encoding
data = pd.get_dummies(data) 
# 一開始Pclass 被偵測為數值，所以沒有被one hot encoding
# Pclass雖然是類別，但是是具大小關係的的類別項目，所以可以不做one hot encoding
# 但是根據老師之前做完的結果，發現這個欄位還是要做one hot encoding準確度會比較高
# 因此這邊使用 get_dummies(data, columns=[要放一個列表]) 的方式，data針對Pclass這個欄位進行one hot encoding
data = pd.get_dummies(data, columns=['Pclass'])
data

,Age,SibSp,Parch,Ticket,Fare,Name_Master,Name_Miss,Name_Mr,Name_Mrs,Sex_female,Sex_male,Cabin_A,Cabin_B,Cabin_C,Cabin_D,Cabin_E,Cabin_F,Cabin_G,Cabin_T,Embarked_C,Embarked_Q,Embarked_S,Pclass_1,Pclass_2,Pclass_3
0,22.0,1,0,1,7.2500,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1
1,38.0,1,0,2,71.2833,0,0,0,1,1,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0
2,26.0,0,0,1,7.9250,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1
3,35.0,1,0,2,53.1000,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0
4,35.0,0,0,1,8.0500,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1304,28.0,0,0,1,8.0500,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1
1305,39.0,0,0,3,108.9000,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0
1306,38.5,0,0,1,7.2500,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1
1307,28.0,0,0,1,8.0500,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1


In [ ]:
# 因為同行直系血親跟旁系血親其實就等於家庭成員的數量，因此新增一個把他們相加後增加一個Family member的欄位
# 但這件事其實是模型建立完後，
data['Family'] = data['SibSp'] + data['Parch']
data

,Age,SibSp,Parch,Ticket,Fare,Name_Master,Name_Miss,Name_Mr,Name_Mrs,Sex_female,Sex_male,Cabin_A,Cabin_B,Cabin_C,Cabin_D,Cabin_E,Cabin_F,Cabin_G,Cabin_T,Embarked_C,Embarked_Q,Embarked_S,Pclass_1,Pclass_2,Pclass_3,Family
0,22.0,1,0,1,7.2500,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1
1,38.0,1,0,2,71.2833,0,0,0,1,1,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,1
2,26.0,0,0,1,7.9250,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0
3,35.0,1,0,2,53.1000,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,1
4,35.0,0,0,1,8.0500,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1304,28.0,0,0,1,8.0500,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0
1305,39.0,0,0,3,108.9000,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0
1306,38.5,0,0,1,7.2500,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0
1307,28.0,0,0,1,8.0500,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0
